In [5]:
import os
import json


train_json = json.load(open(os.path.expanduser('~/data/ShareGPT_Vicuna_unfiltered/shareGPT-llama3-8B.json'), "r"))
len(train_json)

68623

In [6]:
import transformers

from medusa.train.train_legacy import ModelArguments, TrainingArguments


model_args = ModelArguments(
	model_name_or_path=os.path.expanduser('~/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6'),
)

training_args = TrainingArguments(
	output_dir='./train/test',
	medusa_num_heads=5,
	medusa_num_layers=1,
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=training_args.cache_dir,
    model_max_length=training_args.model_max_length,
    padding_side="right",
    use_fast=True,
)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PreTrainedTokenizerFast(name_or_path='/home/camus/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6', vocab_size=128000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_w

In [8]:
import torch


conversations = []
prompts = []
# # import pdb; pdb.set_trace()
for i, conversation in enumerate(train_json[:1]):
	prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
	prompts.append(prompt)
	conversations.append(conversation)

prompts, conversations

(['<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nSummarize the main ideas of Jeff Walker\'s Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients...<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere\'s a summary of the main ideas of Jeff Walker\'s Product Launch Formula, tailored to a growth marketing agency implementing these strategies and tactics for their clients:\n\n**Pre-Launch Phase**\n\n* Identify a problem or pain point in your client\'s target audience and create a solution (product or service) that addresses it\n* Develop a unique value proposition (UVP) highlighting the benefits and features of the solution\n* Create a series of pre-launch content (email series, videos, blog posts, etc.) to build anticipation and educate the audience about the solution\n\n**The 7-Step Launch Framework**\n\n* **Step 1: The Problem Agitator**: Create content that highlights 

In [11]:
tokenizer.pad_token, tokenizer.unk_token, tokenizer.eos_token

(None, None, '<|end_of_text|>')

In [12]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:

# Tokenize conversations
encoding = tokenizer(
	prompts,
	return_tensors="pt",
	padding="max_length",
	truncation=True,
	return_offsets_mapping=True,
)

encoding.input_ids

tensor([[128000, 128000, 128006,  ...,    488,  96618,   2522]])

In [15]:
# Set everything to be ignored, except the assistant part
targets = torch.full_like(encoding.input_ids, -100)

## Mask targets. Only compute loss on the assistant outputs.
#for conv_index, (conversation, target, prompt) in enumerate(zip(conversations, targets, prompts)):
#
#	for turn in conversation:
#		if turn["role"] == "assistant":
#			content = turn["content"]
#			# Unfortunate strip() necessary because chat templates are doing the same.
#			start = prompt.index(content.strip())
#			stop = start + len(content)
#			indices= []
#			for tok_index, (tok_start, tok_stop) in enumerate(encoding.offset_mapping[conv_index]):
#				if tok_stop >= start or tok_start < tok_stop:
#					indices.append(tok_index)
#			target[indices] = encoding.input_ids[conv_index][indices]
#
#target

In [16]:
conv_index, (conversation, target, prompt) = next(enumerate(zip(conversations, targets, prompts)))
conv_index, (conversation, target, prompt)

(0,
 ([{'role': 'user',
    'content': "Summarize the main ideas of Jeff Walker's Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients..."},
   {'role': 'assistant',
    'content': "Here's a summary of the main ideas of Jeff Walker's Product Launch Formula, tailored to a growth marketing agency implementing these strategies and tactics for their clients:\n\n**Pre-Launch Phase**\n\n* Identify a problem or pain point in your client's target audience and create a solution (product or service) that addresses it\n* Develop a unique value proposition (UVP) highlighting the benefits and features of the solution\n* Create a series of pre-launch content (email series, videos, blog posts, etc.) to build anticipation and educate the audience about the solution\n\n**The 7-Step Launch Framework**\n\n* **Step 1: The Problem Agitator**: Create content that highlights the problem or pain point, making the audi

In [17]:
turn = conversation[1]
turn

{'role': 'assistant',
 'content': "Here's a summary of the main ideas of Jeff Walker's Product Launch Formula, tailored to a growth marketing agency implementing these strategies and tactics for their clients:\n\n**Pre-Launch Phase**\n\n* Identify a problem or pain point in your client's target audience and create a solution (product or service) that addresses it\n* Develop a unique value proposition (UVP) highlighting the benefits and features of the solution\n* Create a series of pre-launch content (email series, videos, blog posts, etc.) to build anticipation and educate the audience about the solution\n\n**The 7-Step Launch Framework**\n\n* **Step 1: The Problem Agitator**: Create content that highlights the problem or pain point, making the audience aware of the issue and its impact on their lives\n* **Step 2: The Solution Presenter**: Introduce the solution and its benefits, showcasing how it addresses the problem\n* **Step 3: The Content Packer**: Provide valuable content (e.g.,

In [18]:
content = turn["content"]
# Unfortunate strip() necessary because chat templates are doing the same.
start = prompt.index(content.strip())
stop = start + len(content)
indices= []

start, stop

(306, 3165)

In [20]:
encoding.offset_mapping[conv_index].shape, encoding.offset_mapping[conv_index]

(torch.Size([2048, 2]),
 tensor([[    0,     0],
         [    0,    17],
         [   17,    36],
         ...,
         [10602, 10605],
         [10605, 10605],
         [10608, 10611]]))

In [34]:
encoding.offset_mapping.shape, encoding.offset_mapping[conv_index].tolist()

(torch.Size([1, 2048, 2]),
 [[0, 0],
  [0, 17],
  [17, 36],
  [36, 36],
  [40, 57],
  [57, 57],
  [59, 62],
  [62, 65],
  [65, 68],
  [68, 68],
  [72, 72],
  [77, 77],
  [83, 83],
  [86, 86],
  [91, 91],
  [98, 98],
  [100, 100],
  [108, 108],
  [115, 115],
  [123, 123],
  [128, 128],
  [135, 135],
  [142, 142],
  [145, 145],
  [148, 153],
  [153, 157],
  [157, 157],
  [160, 160],
  [162, 162],
  [169, 169],
  [179, 179],
  [186, 186],
  [199, 199],
  [205, 205],
  [216, 216],
  [220, 220],
  [228, 228],
  [232, 232],
  [238, 238],
  [246, 246],
  [249, 259],
  [259, 278],
  [278, 278],
  [287, 304],
  [304, 304],
  [306, 306],
  [310, 310],
  [312, 312],
  [314, 314],
  [322, 322],
  [325, 325],
  [329, 329],
  [334, 334],
  [340, 340],
  [343, 343],
  [348, 348],
  [355, 355],
  [357, 357],
  [365, 365],
  [372, 372],
  [380, 380],
  [381, 381],
  [390, 390],
  [393, 393],
  [395, 395],
  [402, 402],
  [412, 412],
  [419, 419],
  [432, 432],
  [438, 438],
  [449, 449],
  [453, 453],


In [50]:
[tokenizer._tokenizer.id_to_token(id) for id in encoding.input_ids[0]]

['<|begin_of_text|>',
 '<|begin_of_text|>',
 '<|start_header_id|>',
 'user',
 '<|end_header_id|>',
 'ĊĊ',
 'Sum',
 'mar',
 'ize',
 'Ġthe',
 'Ġmain',
 'Ġideas',
 'Ġof',
 'ĠJeff',
 'ĠWalker',
 "'s",
 'ĠProduct',
 'ĠLaunch',
 'ĠFormula',
 'Ġinto',
 'Ġbullet',
 'Ġpoints',
 'Ġas',
 'Ġit',
 'Ġpert',
 'ains',
 'Ġto',
 'Ġa',
 'Ġgrowth',
 'Ġmarketing',
 'Ġagency',
 'Ġimplementing',
 'Ġthese',
 'Ġstrategies',
 'Ġand',
 'Ġtactics',
 'Ġfor',
 'Ġtheir',
 'Ġclients',
 '...',
 '<|eot_id|>',
 '<|start_header_id|>',
 'assistant',
 '<|end_header_id|>',
 'ĊĊ',
 'Here',
 "'s",
 'Ġa',
 'Ġsummary',
 'Ġof',
 'Ġthe',
 'Ġmain',
 'Ġideas',
 'Ġof',
 'ĠJeff',
 'ĠWalker',
 "'s",
 'ĠProduct',
 'ĠLaunch',
 'ĠFormula',
 ',',
 'Ġtailored',
 'Ġto',
 'Ġa',
 'Ġgrowth',
 'Ġmarketing',
 'Ġagency',
 'Ġimplementing',
 'Ġthese',
 'Ġstrategies',
 'Ġand',
 'Ġtactics',
 'Ġfor',
 'Ġtheir',
 'Ġclients',
 ':ĊĊ',
 '**',
 'Pre',
 '-L',
 'aunch',
 'ĠPhase',
 '**ĊĊ',
 '*',
 'ĠIdentify',
 'Ġa',
 'Ġproblem',
 'Ġor',
 'Ġpain',
 'Ġpoint',


In [31]:
it = enumerate(encoding.offset_mapping[conv_index])

In [33]:
tok_index, (tok_start, tok_stop) = next(it)
tok_index, (tok_start, tok_stop), (tok_stop >= start or tok_start < tok_stop)

(1, (tensor(0), tensor(17)), tensor(True))

In [35]:
indices= []
for tok_index, (tok_start, tok_stop) in enumerate(encoding.offset_mapping[conv_index]):
	if tok_stop >= start or tok_start < tok_stop:
		indices.append(tok_index)

indices

[1,
 2,
 4,
 6,
 7,
 8,
 24,
 25,
 40,
 41,
 43,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212

In [36]:
target[indices] = encoding.input_ids[conv_index][indices]
target

tensor([  -100, 128000, 128006,  ...,    488,  96618,   2522])

In [38]:
list(zip(encoding.input_ids[0], target))

[(tensor(128000), tensor(-100)),
 (tensor(128000), tensor(128000)),
 (tensor(128006), tensor(128006)),
 (tensor(882), tensor(-100)),
 (tensor(128007), tensor(128007)),
 (tensor(271), tensor(-100)),
 (tensor(9370), tensor(9370)),
 (tensor(5730), tensor(5730)),
 (tensor(553), tensor(553)),
 (tensor(279), tensor(-100)),
 (tensor(1925), tensor(-100)),
 (tensor(6848), tensor(-100)),
 (tensor(315), tensor(-100)),
 (tensor(12149), tensor(-100)),
 (tensor(23074), tensor(-100)),
 (tensor(596), tensor(-100)),
 (tensor(5761), tensor(-100)),
 (tensor(24083), tensor(-100)),
 (tensor(31922), tensor(-100)),
 (tensor(1139), tensor(-100)),
 (tensor(17889), tensor(-100)),
 (tensor(3585), tensor(-100)),
 (tensor(439), tensor(-100)),
 (tensor(433), tensor(-100)),
 (tensor(18713), tensor(18713)),
 (tensor(1771), tensor(1771)),
 (tensor(311), tensor(-100)),
 (tensor(264), tensor(-100)),
 (tensor(6650), tensor(-100)),
 (tensor(8661), tensor(-100)),
 (tensor(9266), tensor(-100)),
 (tensor(25976), tensor(-100)